### Setup Libraries

In [2]:
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSD
import prody as prd # really annoying to install. Have to put directly into library folder.
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
from numpy import * 
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline


c:\Users\sodiu\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
run_types = ['WT_v11','G196F_v1'] # add the name of each 
run_count = ['Run1','Run2','Run3'] # add the name of each 
frames = ["{:04}".format(i) for i in range(1,401)] # change last number based on number of frames + 1
sim_length = 200 #how long the simulation is in nanoseconds

#### Finds different in H-Bond percentage
*Will take in 2 VMD H-bond outputs, and then create a file that compares H-bond percent of each interaction*

File has format of: 2 interacting residues, [WT percent, mutated percent, change in percent (WT --> mutated)], [atoms interacting in WT [acceptor_acceptor], atoms interacting in mutant [acceptor_acceptor]]

In [3]:
# take in the 2 results and read in results as a dictionary (key: residues interaction, value: percent)
base_percents = {}

for run_type in run_types: # for each run_type
    base_percents[run_type] = {}
    
    file_path = f"D:\\Research Lab\\MD Sim\\completed_md\\data_storage\\WT_G196F\\SAH_h_bonds\\{run_type.split("_")[0]}\\allhb_detailed35_30.dat"
    with open(file_path, "r") as w:
        data = w.read().split("\n")[2:] # take each line in H-bond file 
        for line in data: # fpr every line in H-bond file
            split_line = line.split(" ") # split line based on space
            split_line = [item for item in split_line if not (item == "" or item == "\t")] # remove spaces / tabs
            proc_line = []
            if(len(split_line) == 3): # for each line that is correct, split each word with "-"
                for word in split_line:
                    proc_line.append(word.split("-"))
                
                t_reslist = sorted([proc_line[0], proc_line[1]], key=lambda x: x[0], reverse=True)
                if(f"{t_reslist[0][0]}_{t_reslist[1][0]}" not in base_percents[run_type].keys()):
                    base_percents[run_type][f"{t_reslist[0][0]}_{t_reslist[1][0]}"] = {}
                base_percents[run_type][f"{t_reslist[0][0]}_{t_reslist[1][0]}"][f"{proc_line[0][2]}_{proc_line[1][2]}"] = float(proc_line[2][0][0:-1])

print(base_percents)
# create a union of keys
key_union = []
for run_type in base_percents:
    key_union.extend(base_percents[run_type].keys())

key_union = set(key_union)

# Find difference between each. Create new list
compared = []
for key in key_union:
    compared.append([key])
    for run_type in base_percents:
        try:
            compared[-1].append(float(round(sum(list(base_percents[run_type][key].values())), 2)))
            compared[-1].append(base_percents[run_type][key])
        except:
            compared[-1].append(0)
            compared[-1].append({"N/A":0})
    compared[-1].append(float(round(compared[-1][3] - compared[-1][1], 2))) # second (mutated) - first (WT). Changed WT --> mutated
    

# save dictionary to file
file = f"D:\\Research Lab\\MD Sim\\completed_md\\Plots\\{run_types[0].split("_")[0]}_{run_types[1].split("_")[0]}_h_bonds.txt"
with open(file, "w") as w:
    compared = sorted(compared, key=lambda x: abs(x[-1]), reverse=True)

    for i, values in enumerate(compared):
        w.write(f"{values[0]}: {values[1::2]}  {values[2::2]}\n")


print(compared)


{'WT_v11': {'SAH1_ALA461': {'N_O4': 26.42, 'O3_O': 0.02, 'N_N5': 0.02}, 'SER484_SAH1': {'N_O2': 8.13, 'OG_N3': 13.98, 'O3_O': 9.07, 'O4_O': 30.32, 'O4_OG': 0.1, 'N_N3': 1.4, 'N_O4': 0.25, 'N1_O': 0.3}, 'THR481_SAH1': {'OG1_N2': 31.9, 'N4_O': 3.73, 'OG1_N4': 0.02}, 'THR464_SAH1': {'O4_OG1': 28.42, 'O3_OG1': 2.95, 'OG1_O4': 26.28, 'OG1_N3': 0.3, 'O4_N': 0.13, 'OG1_N5': 0.02, 'OG1_O3': 0.4, 'O5_OG1': 0.02, 'N_O4': 0.02}, 'TRP349_SAH1': {'O5_N': 0.03}, 'SAH1_PRO348': {'O5_O': 7.25, 'N1_O': 5.03}, 'SAH1_ASN456': {'N4_OD1': 26.97, 'ND2_N5': 17.8, 'N4_ND2': 0.15, 'ND2_O1': 3.5, 'ND2_N1': 0.28, 'ND2_O5': 0.15, 'N1_OD1': 0.57, 'N1_ND2': 0.02, 'O5_OD1': 0.03}, 'SAH1_GLU463': {'O5_OE2': 0.37, 'O4_OE2': 0.35, 'O5_OE1': 0.68, 'O3_OE2': 6.55, 'O4_OE1': 0.52, 'O3_OE1': 6.57, 'O4_O': 0.03, 'N1_OE1': 1.07, 'N1_OE2': 0.68, 'N_O1': 0.32}, 'SAH1_ALA368': {'N1_O': 0.27, 'O5_O': 0.7}, 'THR573_SAH1': {'O5_OG1': 0.32, 'OG1_O1': 0.62, 'OG1_N1': 0.1}, 'SAH1_ARG345': {'NH2_N1': 0.02, 'NH1_O1': 2.07, 'NH1_N1': 3.

#### Create Inputs for Main File
*Create what can go in the setup section so that every H-bond is accounted for automatically*

In [6]:
"""Data need for each residue:
   For each RPI: (1) determine index / name of acceptor (2) determine atom that is acceptor (3) determine atom that is acceptor
"""
amino_acids_3_letter_codes = [
    "ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY", "HIS", "ILE",
    "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL",
    "HSE", "HSD"
]


def extract_res_data(interactor):
   others = interactor[0]
   atom = interactor[1]
   index = ""
   res_type = ""
   for letter in others:
      if letter.isnumeric():
         index = index + letter
      else:
         res_type = res_type + letter
   return index + "_" + res_type + "_" + atom

interactions = {}

# read in the H-bond data. 
for run_type in run_types: # for each run_type
    
    file_path = f"F:\\Research Lab\\MD Sim\\completed_md\\data_storage\\WT_G196F\\SAH_h_bonds\\{run_type.split("_")[0]}\\allhb_detailed35_30.dat"
    with open(file_path, "r") as w:
        data = w.read().split("\n")[2:]
        for line in data: # for every line in H-bond file

            # figure out the 2 residues interacting
            split_line = line.split(" ") # split line based on space
            split_line = [item for item in split_line if not (item == "" or item == "\t")] # makes 1 = donor, 2 = acceptor
            if(len(split_line) > 2):
               donor = split_line[0].split("-")[::2]
               acceptor = split_line[1].split("-")[::2]
               
               # create interaction name
               if(donor > acceptor):
                  interaction_name = acceptor[0] + "_" + donor[0]
               else:
                  interaction_name = donor[0] + "_" + acceptor[0]

               # extract index, res and atom for both
               donor_data = extract_res_data(donor)
               acceptor_data = extract_res_data(acceptor)

               # put together
               if interaction_name not in interactions.keys():
                  interactions[interaction_name] = set()
                
               interactions[interaction_name].add(donor_data + "-" + acceptor_data)


# go through each interaction and create input
toPrint = []
for interaction in interactions.keys():
   values = interactions[interaction]

   # 1) assign each acceptor / donor a variable
   donor_dict = {}
   dValue = 1
   acc_dict = {}
   aValue = 1
   for pair in values:
      donor = pair.split("-")[0] # donor
      if donor not in donor_dict.keys():
         donor_dict[donor] = "d" + str(dValue)
         dValue = dValue + 1
      
      acceptor = pair.split("-")[1] # acceptor
      if acceptor not in acc_dict.keys():
         acc_dict[acceptor] = "a" + str(aValue)
         aValue = aValue + 1
   
   # 2) create list of each donor-acceptor pair using variable
   pair_list = []
   for pair in values:
      donor = pair.split("-")[0]
      acceptor = pair.split("-")[1]

      pair_list.append([donor_dict[donor],acc_dict[acceptor]])

   # 3) create atom variable assignmnet
   for donor_data in donor_dict.keys(): # donor
      donor_var = donor_dict[donor_data]
      donor_data = donor_data.split("_") # index + "_" + res_type + "_" + atom
      if donor_data[1] in amino_acids_3_letter_codes: # is an amino acid
         Hval = f"H_{donor_data[2]}"
         if(donor_data[2] == "N"):
            Hval = "HN"
         toPrint.append(f'{donor_var} = [{donor_data[0]}, "{donor_data[2]}", "{Hval}"]')
      else: # is a ligand
         Hval = f"H_{donor_data[2]}"
         toPrint.append(f'{donor_var} = [["{donor_data[1]}", {donor_data[0]}], "{donor_data[2]}", "{Hval}"]')
   
   toPrint.append("")

   for acceptor_data in acc_dict.keys(): # acceptor
      acceptor_var = acc_dict[acceptor_data]
      acceptor_data = acceptor_data.split("_") # index + "_" + res_type + "_" + atom
      if acceptor_data[1] in amino_acids_3_letter_codes: # is an amino acid
         toPrint.append(f'{acceptor_var} = [{acceptor_data[0]}, "{acceptor_data[2]}"]')
      else: # is a ligand
         toPrint.append(f'{acceptor_var} = [["{acceptor_data[1]}", {acceptor_data[0]}], "{acceptor_data[2]}"]')

   toPrint.append("")
   
   # 4) rpiH
   toPrint.append(f"rpiHs = {str(pair_list).replace("'","")}")
   toPrint.append("")
   toPrint.append("")

# 5) save to a file
file = f"F:\\Research Lab\\MD Sim\\completed_md\\Plots\\{run_types[0].split("_")[0]}_{run_types[1].split("_")[0]}_h_inputs.txt"
with open(file, "w") as f:
   for ind, line in enumerate(toPrint):
      if(ind != 0):
         f.write("\n")
      f.write(line)

#### FAD - Complete Inputs

In [ ]:
### FAD1_ASP210
d1 = [["FAD", 1], "O12", "H20"]
d2 = [["FAD", 1], "O11", "H17"]

a1 = [210, "OD1"]
a2 = [210, "OD2"]

rpiHs = [[d1, a1], [d1, a2], [d2, a2], [d2, a1]]




### FAD1_ALA155
d1 = [["FAD", 1], "O1", "H1"]

a1 = [155, "O"]

rpiHs = [[d1, a1]]



### FAD1_ALA195
d1 = [195, "N", "HN"]
d2 = [["FAD", 1], "O1", "H1"]

a1 = [["FAD", 1], "O4"]
a2 = [["FAD", 1], "O2"]
a3 = [["FAD", 1], "O1"]
a4 = [195, "O"]
a5 = [["FAD", 1], "O8"]

rpiHs = [[d1, a1], [d1, a2], [d1, a3], [d2, a4], [d1, a5]]




### FAD1_ARG157
d1 = [["FAD", 1], "O5", "H5"]
d2 = [157, "NH2", "HH21", "HH22"]
d3 = [["FAD", 1], "O10", "H16"]
d4 = [157, "N", "HN"]
d5 = [157, "NE", "HE"]
d6 = [["FAD", 1], "O6", "H7"]
d7 = [["FAD", 1], "N6", "H25", "H26"]
d8 = [157, "NH1", "HH12", "HH11"]

a1 = [157, "NH2"]
a2 = [["FAD", 1], "N6"]
a3 = [157, "NE"]
a4 = [["FAD", 1], "O2"]
a5 = [["FAD", 1], "O8"]
a6 = [157, "NH1"]
a7 = [["FAD", 1], "O1"]
a8 = [["FAD", 1], "O10"]
a9 = [["FAD", 1], "N7"]
a10 = [["FAD", 1], "N2"]
a11 = [["FAD", 1], "O5"]
a12 = [["FAD", 1], "O4"]
a13 = [["FAD", 1], "O6"]
a14 = [["FAD", 1], "N4"]
a15 = [["FAD", 1], "O7"]

rpiHs = [[d1, a1], [d2, a2], [d3, a3], [d4, a4], [d5, a5], [d6, a6], [d5, a7], [d5, a8], [d7, a3], [d4, a7], [d2, a9], [d8, a10], [d8, a9], [d5, a11], [d2, a5], [d4, a5], [d4, a12], [d8, a2], [d2, a10], [d2, a13], [d5, a9], [d5, a10], [d7, a6], [d6, a1], [d3, a6], [d4, a11], [d2, a14], [d4, a15], [d7, a1], [d2, a11]]




### FAD1_ASP159
d1 = [["FAD", 1], "N1", "H13"]
d2 = [159, "N", "HN"]
d3 = [["FAD", 1], "O6", "H7"]
d4 = [["FAD", 1], "O7", "H13"]
d5 = [["FAD", 1], "N3", "H19"]
d6 = [["FAD", 1], "O5", "H5"]
d7 = [["FAD", 1], "O1", "H1"]

a1 = [159, "OD2"]
a2 = [159, "OD1"]
a3 = [["FAD", 1], "O13"]
a4 = [["FAD", 1], "N1"]
a5 = [159, "O"]
a6 = [159, "N"]
a7 = [["FAD", 1], "O7"]
a8 = [["FAD", 1], "O9"]

rpiHs = [[d1, a1], [d1, a2], [d2, a3], [d3, a2], [d2, a4], [d4, a2], [d1, a5], [d5, a5], [d3, a6], [d6, a1], [d7, a2], [d4, a1], [d2, a7], [d3, a1], [d5, a2], [d4, a5], [d6, a2], [d2, a8]]




### LEU271_FAD1
d1 = [["FAD", 1], "O5", "H5"]
d2 = [["FAD", 1], "N1", "H13"]
d3 = [["FAD", 1], "O6", "H7"]

a1 = [271, "O"]

rpiHs = [[d1, a1], [d2, a1], [d3, a1]]





### LYS217_FAD1
d1 = [217, "NZ", "HZ1", "HZ2", "HZ3"]

a1 = [["FAD", 1], "O12"]
a2 = [["FAD", 1], "O8"]
a3 = [["FAD", 1], "O15"]
a4 = [["FAD", 1], "O11"]
a5 = [["FAD", 1], "O2"]
a6 = [["FAD", 1], "N7"]

rpiHs = [[d1, a1], [d1, a2], [d1, a3], [d1, a4], [d1, a5], [d1, a6]]




### HSD201_FAD1
d1 = [["FAD", 1], "O12", "H20"]
d2 = [["FAD", 1], "O11", "H17"]
d3 = [201, "ND1", "HD1"]
d4 = [["FAD", 1], "O1", "H1"]
d5 = [["FAD", 1], "O5", "H5"]
d6 = [201, "N", "HN"]
d7 = [["FAD", 1], "O10", "H16"]
d8 = [["FAD", 1], "O6", "H7"]

a1 = [201, "NE2"]
a2 = [201, "ND1"]
a3 = [201, "O"]
a4 = [["FAD", 1], "O12"]
a5 = [["FAD", 1], "O11"]
a6 = [["FAD", 1], "O4"]
a7 = [["FAD", 1], "O14"]
a8 = [["FAD", 1], "N4"]
a9 = [["FAD", 1], "O5"]
a10 = [["FAD", 1], "O6"]

rpiHs = [[d1, a1], [d1, a2], [d2, a1], [d2, a3], [d3, a4], [d4, a1], [d5, a2], [d6, a4], [d6, a5], [d3, a6], [d5, a1], [d3, a7], [d3, a8], [d3, a9], [d7, a1], [d1, a3], [d3, a10], [d8, a2], [d8, a1], [d3, a5]]





### TRP95_FAD1
d1 = [95, "N", "HN"]
d2 = [["FAD", 1], "N3", "H19"]

a1 = [["FAD", 1], "O13"]
a2 = [95, "O"]

rpiHs = [[d1, a1], [d2, a2]]





### GLU203_FAD1
d1 = [["FAD", 1], "O1", "H1"]
d2 = [["FAD", 1], "O11", "H17"]
d3 = [["FAD", 1], "N6", "H25","H26"]
d4 = [203, "N", "HN"]
d5 = [["FAD", 1], "O5", "H5"]
d6 = [["FAD", 1], "O6", "H7"]
d7 = [["FAD", 1], "O10", "H16"]
d8 = [["FAD", 1], "O12", "H29"]

a1 = [203, "OE2"]
a2 = [203, "O"]
a3 = [203, "OE1"]
a4 = [["FAD", 1], "O12"]
a5 = [["FAD", 1], "O5"]
a6 = [["FAD", 1], "O11"]

rpiHs = [[d1, a1], [d2, a2], [d3, a3], [d1, a3], [d3, a1], [d2, a1], [d4, a4], [d4, a5], [d4, a6], [d5, a1], [d6, a3], [d5, a3], [d7, a3], [d6, a1], [d7, a1], [d2, a3], [d8, a3], [d8, a1], [d8, a2]]


#### FAD - % for each residue interaction

In [ ]:
FAD1_ASP210: [143.05, 53.37, -89.68]  [{'O12_OD1': 50.33, 'O11_OD1': 19.65, 'O12_OD2': 36.07, 'O11_OD2': 37.0}, {'O11_OD1': 9.25, 'O12_OD2': 13.3, 'O12_OD1': 25.02, 'O11_OD2': 5.8}]
FAD1_ALA155: [60.25, 0, -60.25]  [{'O1_O': 60.25}, {'N/A': 0}]
FAD1_ALA195: [46.69, 0.03, -46.66]  [{'N_O1': 4.42, 'N_O2': 42.1, 'N_O8': 0.15, 'N_O4': 0.02}, {'O1_O': 0.03}]
FAD1_ARG157: [56.39, 22.75, -33.64]  [{'N_O5': 43.8, 'N_O7': 0.55, 'N_O4': 0.27, 'N_O1': 1.78, 'N_O2': 0.85, 'NH2_O5': 0.03, 'N_O8': 8.92, 'NE_O8': 0.07, 'NH2_O8': 0.02, 'NH1_N6': 0.03, 'O6_NH1': 0.02, 'NE_O5': 0.03, 'O10_NE': 0.02}, {'N_O5': 7.1, 'N_O1': 10.0, 'O10_NH1': 0.2, 'NH2_O6': 0.02, 'O5_NH2': 0.02, 'O6_NH2': 0.02, 'N_O2': 3.5, 'N_O8': 0.15, 'NH1_N2': 0.68, 'N6_NH2': 0.12, 'NH2_N2': 0.27, 'NE_N2': 0.18, 'NE_O1': 0.07, 'NH2_N4': 0.02, 'NE_O10': 0.03, 'N6_NH1': 0.03, 'NH2_N6': 0.03, 'N6_NE': 0.02, 'NH2_N7': 0.15, 'NE_N7': 0.02, 'NH1_N6': 0.05, 'NH1_N7': 0.07}]
FAD1_ASP159: [100.36, 74.1, -26.26]  [{'N1_OD2': 19.95, 'N_O9': 28.78, 'N1_OD1': 17.27, 'O6_OD1': 2.53, 'O6_OD2': 2.5, 'N3_OD1': 0.02, 'N_N1': 0.12, 'O5_OD1': 0.27, 'O5_OD2': 0.3, 'O7_OD1': 13.0, 'O7_OD2': 15.62}, {'N_O9': 14.97, 'O6_OD2': 4.65, 'O6_OD1': 4.77, 'N1_OD2': 2.67, 'N1_O': 2.4, 'O5_OD2': 7.67, 'O5_OD1': 8.27, 'N3_O': 2.22, 'N_O7': 1.15, 'O7_OD2': 9.78, 'O7_OD1': 12.38, 'N_O13': 0.07, 'N1_OD1': 2.83, 'O1_OD1': 0.02, 'N_N1': 0.03, 'O7_O': 0.2, 'O6_N': 0.02}]
LEU271_FAD1: [0.07, 25.56, 25.49]  [{'N1_O': 0.07}, {'N1_O': 13.52, 'O6_O': 11.92, 'O5_O': 0.12}]
LYS217_FAD1: [30.69, 6.01, -24.68]  [{'NZ_O2': 1.37, 'NZ_O15': 25.88, 'NZ_O8': 0.77, 'NZ_O12': 2.62, 'NZ_O11': 0.05}, {'NZ_O2': 4.12, 'NZ_O15': 0.05, 'NZ_O12': 0.62, 'NZ_O8': 0.88, 'NZ_O11': 0.22, 'NZ_N7': 0.12}]
HSD201_FAD1: [10.13, 33.35, 23.22]  [{'O5_NE2': 0.02, 'O11_NE2': 6.3, 'O6_NE2': 0.02, 'N_O12': 0.03, 'O12_NE2': 0.28, 'O10_NE2': 0.23, 'O1_NE2': 1.15, 'ND1_O14': 0.13, 'ND1_O12': 0.02, 'ND1_O11': 1.95}, {'ND1_O12': 0.5, 'O10_NE2': 0.42, 'ND1_O6': 1.25, 'O5_NE2': 4.2, 'O6_NE2': 2.17, 'ND1_O14': 0.32, 'ND1_N4': 0.52, 'N_O12': 9.53, 'N_O11': 6.97, 'O12_NE2': 0.05, 'ND1_O11': 0.08, 'O1_NE2': 0.22, 'O11_O': 2.95, 'ND1_O5': 1.63, 'ND1_O4': 1.62, 'O5_ND1': 0.03, 'O6_ND1': 0.1, 'O12_O': 0.77, 'O12_ND1': 0.02}]
TRP95_FAD1: [114.3, 93.51, -20.79]  [{'N3_O': 59.02, 'N_O13': 55.28}, {'N3_O': 49.08, 'N_O13': 44.43}]
GLU203_FAD1: [16.02, 36.5, 20.48]  [{'O11_OE2': 0.02, 'O11_OE1': 0.02, 'O10_OE2': 0.05, 'O10_OE1': 0.03, 'O1_OE2': 5.63, 'O5_OE2': 2.6, 'O1_OE1': 4.75, 'O5_OE1': 2.83, 'N_O5': 0.02, 'O6_OE1': 0.05, 'O6_OE2': 0.02}, {'O11_O': 1.72, 'O12_O': 0.15, 'O10_OE2': 5.05, 'O10_OE1': 3.08, 'N_O12': 0.15, 'O11_OE2': 0.38, 'O11_OE1': 0.28, 'O12_OE2': 0.08, 'N6_OE1': 0.02, 'O1_OE2': 6.5, 'O5_OE2': 3.83, 'O1_OE1': 9.18, 'O5_OE1': 5.73, 'N_O11': 0.3, 'N6_OE2': 0.02, 'O12_OE1': 0.03}]
FAD1_ALA175: [6.8, 23.38, 16.58]  [{'N_O10': 1.15, 'N_O8': 0.15, 'N_N2': 5.47, 'N_O2': 0.03}, {'N_O8': 23.3, 'N_O10': 0.08}]
FAD1_ASN173: [15.3, 0.48, -14.82]  [{'N6_O': 15.3}, {'N6_O': 0.3, 'N6_OD1': 0.18}]
HSD127_FAD1: [6.89, 21.11, 14.22]  [{'ND1_O13': 6.87, 'ND1_N5': 0.02}, {'ND1_O13': 21.08, 'ND1_N5': 0.03}]
GLY158_FAD1: [14.04, 25.81, 11.77]  [{'N_O5': 5.4, 'N_O7': 3.05, 'N_O1': 3.47, 'N_O2': 2.12}, {'N_O5': 5.48, 'N_O7': 1.3, 'O7_O': 0.07, 'N_O1': 1.02, 'N_O2': 14.77, 'O5_O': 0.27, 'O6_O': 2.88, 'O1_O': 0.02}]
LYS273_FAD1: [0, 7.25, 7.25]  [{'N/A': 0}, {'NZ_O9': 0.07, 'NZ_O5': 5.82, 'O11_O': 0.1, 'O12_O': 0.2, 'NZ_O2': 0.22, 'NZ_O6': 0.32, 'O10_O': 0.07, 'NZ_O8': 0.4, 'NZ_N6': 0.02, 'NZ_N2': 0.03}]
TYR197_FAD1: [2.29, 9.31, 7.02]  [{'OH_O6': 1.42, 'O6_OH': 0.07, 'O5_OH': 0.52, 'OH_O2': 0.13, 'OH_O4': 0.13, 'OH_O15': 0.02}, {'OH_O6': 5.7, 'OH_O4': 2.23, 'O5_OH': 1.12, 'O6_OH': 0.07, 'OH_O2': 0.07, 'O7_OH': 0.03, 'OH_O8': 0.02, 'OH_O5': 0.05, 'O1_OH': 0.02}]
FAD1_ALA204: [0, 5.05, 5.05]  [{'N/A': 0}, {'O11_N': 0.08, 'O12_N': 0.02, 'N_O12': 0.15, 'N_O11': 1.77, 'O11_O': 2.18, 'O12_O': 0.85}]
GLN228_FAD1: [0.03, 5.05, 5.02]  [{'NE2_O6': 0.03}, {'NE2_O6': 5.0, 'NE2_O2': 0.05}]
PRO160_FAD1: [0, 4.45, 4.45]  [{'N/A': 0}, {'N3_O': 4.45}]
GLY200_FAD1: [0, 3.35, 3.35]  [{'N/A': 0}, {'O12_O': 3.27, 'O11_O': 0.03, 'O1_O': 0.05}]
GLN164_FAD1: [0, 3.3, 3.3]  [{'N/A': 0}, {'NE2_N2': 0.03, 'NE2_O11': 0.15, 'NE2_N7': 0.15, 'N6_OE1': 0.25, 'N6_NE2': 0.02, 'N_N7': 2.63, 'O11_OE1': 0.03, 'NE2_N6': 0.02, 'O10_OE1': 0.02}]
THR94_FAD1: [1.25, 4.32, 3.07]  [{'OG1_O13': 1.25}, {'OG1_O13': 4.2, 'N3_OG1': 0.05, 'OG1_N5': 0.07}]
HSD213_FAD1: [2.04, 5.05, 3.01]  [{'ND1_O12': 0.27, 'O11_NE2': 0.82, 'O11_ND1': 0.05, 'ND1_O11': 0.65, 'ND1_N2': 0.03, 'ND1_N6': 0.02, 'O12_NE2': 0.2}, {'ND1_O11': 1.05, 'ND1_O12': 0.02, 'O11_ND1': 0.03, 'O11_NE2': 0.07, 'N6_NE2': 3.2, 'N6_ND1': 0.58, 'ND1_N7': 0.1}]
GLU216_FAD1: [2.93, 0.3, -2.63]  [{'N6_OE1': 1.58, 'N6_OE2': 1.35}, {'N6_OE1': 0.1, 'N6_OE2': 0.2}]
GLY205_FAD1: [0, 2.27, 2.27]  [{'N/A': 0}, {'N_N4': 0.08, 'N_N2': 0.07, 'N_N6': 0.02, 'N_O11': 1.65, 'O11_O': 0.35, 'O12_O': 0.1}]
HSE96_FAD1: [0, 1.69, 1.69]  [{'N/A': 0}, {'NE2_O9': 0.02, 'NE2_O13': 1.65, 'NE2_N5': 0.02}]
SER272_FAD1: [0, 1.59, 1.59]  [{'N/A': 0}, {'O6_O': 0.13, 'O12_O': 0.07, 'O11_O': 0.02, 'O1_O': 0.02, 'O10_O': 0.4, 'OG_O2': 0.92, 'OG_O6': 0.03}]
TYR174_FAD1: [0.07, 1.36, 1.29]  [{'OH_N7': 0.02, 'N6_OH': 0.05}, {'OH_N2': 0.18, 'N6_OH': 1.03, 'OH_N7': 0.15}]
FAD1_ARG68: [0, 1.02, 1.02]  [{'N/A': 0}, {'NH2_O9': 0.95, 'NH1_O9': 0.05, 'NH2_N1': 0.02}]
LYS199_FAD1: [0, 0.83, 0.83]  [{'N/A': 0}, {'O12_O': 0.8, 'O11_O': 0.03}]
GLU167_FAD1: [1.27, 0.47, -0.8]  [{'N6_OE1': 0.67, 'N6_OE2': 0.6}, {'N6_OE1': 0.3, 'N6_OE2': 0.17}]
SER206_FAD1: [0.02, 0.73, 0.71]  [{'OG_N4': 0.02}, {'O11_O': 0.43, 'N_O11': 0.15, 'N_N4': 0.08, 'O12_O': 0.07}]
THR129_FAD1: [0, 0.48, 0.48]  [{'N/A': 0}, {'OG1_O9': 0.48}]
VAL176_FAD1: [0.3, 0, -0.3]  [{'N_O10': 0.03, 'N_N2': 0.27}, {'N/A': 0}]
FAD1_ARG132: [0, 0.23, 0.23]  [{'N/A': 0}, {'NH1_O9': 0.08, 'NH2_O9': 0.15}]
GLY162_FAD1: [0, 0.12, 0.12]  [{'N/A': 0}, {'N6_O': 0.12}]
TRP165_FAD1: [0, 0.1, 0.1]  [{'N/A': 0}, {'N6_O': 0.08, 'N_N6': 0.02}]
FAD1_ASP163: [0, 0.07, 0.07]  [{'N/A': 0}, {'O11_OD2': 0.02, 'N6_OD1': 0.03, 'N6_OD2': 0.02}]
FAD1_ALA209: [0.08, 0.03, -0.05]  [{'O11_O': 0.08}, {'O11_O': 0.03}]
GLU166_FAD1: [0, 0.03, 0.03]  [{'N/A': 0}, {'N6_OE2': 0.03}]
FAD1_ASP177: [0, 0.02, 0.02]  [{'N/A': 0}, {'N6_OD2': 0.02}]


#### FAD - Inputs Where Donor is Missing Hydrogen

In [ ]:
d1 = [["FAD", 1], "O12", "H_O12"]
d2 = [["FAD", 1], "O11", "H_O11"]

a1 = [210, "OD1"]
a2 = [210, "OD2"]

rpiHs = [[d1, a1], [d1, a2], [d2, a2], [d2, a1]]


d1 = [["FAD", 1], "O1", "H_O1"]

a1 = [155, "O"]

rpiHs = [[d1, a1]]


d1 = [["FAD", 1], "N1", "H_N1"]
d2 = [159, "N", "HN"]
d3 = [["FAD", 1], "O6", "H_O6"]
d4 = [["FAD", 1], "O7", "H_O7"]
d5 = [["FAD", 1], "N3", "H_N3"]
d6 = [["FAD", 1], "O5", "H_O5"]
d7 = [["FAD", 1], "O1", "H_O1"]

a1 = [159, "OD2"]
a2 = [159, "OD1"]
a3 = [["FAD", 1], "O13"]
a4 = [["FAD", 1], "N1"]
a5 = [159, "O"]
a6 = [159, "N"]
a7 = [["FAD", 1], "O7"]
a8 = [["FAD", 1], "O9"]

rpiHs = [[d1, a1], [d1, a2], [d2, a3], [d3, a2], [d2, a4], [d4, a2], [d1, a5], [d5, a5], [d3, a6], [d6, a1], [d7, a2], [d4, a1], [d2, a7], [d3, a1], [d5, a2], [d4, a5], [d6, a2], [d2, a8]]


d1 = [95, "N", "HN"]
d2 = [["FAD", 1], "N3", "H_N3"]

a1 = [["FAD", 1], "O13"]
a2 = [95, "O"]

rpiHs = [[d1, a1], [d2, a2]]


d1 = [217, "NZ", "H_NZ"]

a1 = [["FAD", 1], "O12"]
a2 = [["FAD", 1], "O8"]
a3 = [["FAD", 1], "O15"]
a4 = [["FAD", 1], "O11"]
a5 = [["FAD", 1], "O2"]
a6 = [["FAD", 1], "N7"]

rpiHs = [[d1, a1], [d1, a2], [d1, a3], [d1, a4], [d1, a5], [d1, a6]]


d1 = [175, "N", "HN"]

a1 = [["FAD", 1], "N2"]
a2 = [["FAD", 1], "O8"]
a3 = [["FAD", 1], "O10"]
a4 = [["FAD", 1], "O2"]

rpiHs = [[d1, a1], [d1, a2], [d1, a3], [d1, a4]]


d1 = [["FAD", 1], "O7", "H_O7"]
d2 = [["FAD", 1], "O6", "H_O6"]
d3 = [["FAD", 1], "O1", "H_O1"]
d4 = [158, "N", "HN"]
d5 = [["FAD", 1], "O5", "H_O5"]

a1 = [158, "O"]
a2 = [["FAD", 1], "O7"]
a3 = [["FAD", 1], "O1"]
a4 = [["FAD", 1], "O2"]
a5 = [["FAD", 1], "O5"]

rpiHs = [[d1, a1], [d2, a1], [d3, a1], [d4, a2], [d5, a1], [d4, a3], [d4, a4], [d4, a5]]


d1 = [127, "ND1", "H_ND1"]

a1 = [["FAD", 1], "N5"]
a2 = [["FAD", 1], "O13"]

rpiHs = [[d1, a1], [d1, a2]]


d1 = [["FAD", 1], "O5", "H_O5"]
d2 = [157, "NH2", "H_NH2"]
d3 = [["FAD", 1], "O10", "H_O10"]
d4 = [157, "N", "HN"]
d5 = [157, "NE", "H_NE"]
d6 = [["FAD", 1], "O6", "H_O6"]
d7 = [["FAD", 1], "N6", "H_N6"]
d8 = [157, "NH1", "H_NH1"]

a1 = [157, "NH2"]
a2 = [["FAD", 1], "N6"]
a3 = [157, "NE"]
a4 = [["FAD", 1], "O2"]
a5 = [["FAD", 1], "O8"]
a6 = [157, "NH1"]
a7 = [["FAD", 1], "O1"]
a8 = [["FAD", 1], "O10"]
a9 = [["FAD", 1], "N7"]
a10 = [["FAD", 1], "N2"]
a11 = [["FAD", 1], "O5"]
a12 = [["FAD", 1], "O4"]
a13 = [["FAD", 1], "O6"]
a14 = [["FAD", 1], "N4"]
a15 = [["FAD", 1], "O7"]

rpiHs = [[d1, a1], [d2, a2], [d3, a3], [d4, a4], [d5, a5], [d6, a6], [d5, a7], [d5, a8], [d7, a3], [d4, a7], [d2, a9], [d8, a10], [d8, a9], [d5, a11], [d2, a5], [d4, a5], [d4, a12], [d8, a2], [d2, a10], [d2, a13], [d5, a9], [d5, a10], [d7, a6], [d6, a1], [d3, a6], [d4, a11], [d2, a14], [d4, a15], [d7, a1], [d2, a11]]


d1 = [195, "N", "HN"]
d2 = [["FAD", 1], "O1", "H_O1"]

a1 = [["FAD", 1], "O4"]
a2 = [["FAD", 1], "O2"]
a3 = [["FAD", 1], "O1"]
a4 = [195, "O"]
a5 = [["FAD", 1], "O8"]

rpiHs = [[d1, a1], [d1, a2], [d1, a3], [d2, a4], [d1, a5]]


d1 = [197, "OH", "H_OH"]
d2 = [["FAD", 1], "O6", "H_O6"]
d3 = [["FAD", 1], "O5", "H_O5"]
d4 = [["FAD", 1], "O7", "H_O7"]
d5 = [["FAD", 1], "O1", "H_O1"]

a1 = [["FAD", 1], "O6"]
a2 = [["FAD", 1], "O4"]
a3 = [197, "OH"]
a4 = [["FAD", 1], "O2"]
a5 = [["FAD", 1], "O5"]
a6 = [["FAD", 1], "O8"]
a7 = [["FAD", 1], "O15"]

rpiHs = [[d1, a1], [d1, a2], [d2, a3], [d3, a3], [d1, a4], [d4, a3], [d1, a5], [d1, a6], [d1, a7], [d5, a3]]


d1 = [["FAD", 1], "N6", "H_N6"]

a1 = [216, "OE2"]
a2 = [216, "OE1"]

rpiHs = [[d1, a1], [d1, a2]]


d1 = [["FAD", 1], "O5", "H_O5"]
d2 = [["FAD", 1], "N1", "H_N1"]
d3 = [["FAD", 1], "O6", "H_O6"]

a1 = [271, "O"]

rpiHs = [[d1, a1], [d2, a1], [d3, a1]]


d1 = [213, "ND1", "H_ND1"]
d2 = [["FAD", 1], "O11", "H_O11"]
d3 = [["FAD", 1], "N6", "H_N6"]
d4 = [["FAD", 1], "O12", "H_O12"]

a1 = [["FAD", 1], "O12"]
a2 = [213, "ND1"]
a3 = [213, "NE2"]
a4 = [["FAD", 1], "O11"]
a5 = [["FAD", 1], "N2"]
a6 = [["FAD", 1], "N7"]
a7 = [["FAD", 1], "N6"]

rpiHs = [[d1, a1], [d2, a2], [d2, a3], [d1, a4], [d1, a5], [d3, a3], [d1, a6], [d4, a3], [d3, a2], [d1, a7]]


d1 = [["FAD", 1], "O12", "H_O12"]
d2 = [["FAD", 1], "O11", "H_O11"]
d3 = [201, "ND1", "H_ND1"]
d4 = [["FAD", 1], "O1", "H_O1"]
d5 = [["FAD", 1], "O5", "H_O5"]
d6 = [201, "N", "HN"]
d7 = [["FAD", 1], "O10", "H_O10"]
d8 = [["FAD", 1], "O6", "H_O6"]

a1 = [201, "NE2"]
a2 = [201, "ND1"]
a3 = [201, "O"]
a4 = [["FAD", 1], "O12"]
a5 = [["FAD", 1], "O11"]
a6 = [["FAD", 1], "O4"]
a7 = [["FAD", 1], "O14"]
a8 = [["FAD", 1], "N4"]
a9 = [["FAD", 1], "O5"]
a10 = [["FAD", 1], "O6"]

rpiHs = [[d1, a1], [d1, a2], [d2, a1], [d2, a3], [d3, a4], [d4, a1], [d5, a2], [d6, a4], [d6, a5], [d3, a6], [d5, a1], [d3, a7], [d3, a8], [d3, a9], [d7, a1], [d1, a3], [d3, a10], [d8, a2], [d8, a1], [d3, a5]]


d1 = [176, "N", "HN"]

a1 = [["FAD", 1], "N2"]
a2 = [["FAD", 1], "O10"]

rpiHs = [[d1, a1], [d1, a2]]


d1 = [["FAD", 1], "O11", "H_O11"]

a1 = [209, "O"]

rpiHs = [[d1, a1]]


d1 = [["FAD", 1], "N3", "H_N3"]
d2 = [94, "OG1", "H_OG1"]

a1 = [94, "OG1"]
a2 = [["FAD", 1], "O13"]
a3 = [["FAD", 1], "N5"]

rpiHs = [[d1, a1], [d2, a2], [d2, a3]]


d1 = [["FAD", 1], "N6", "H_N6"]

a1 = [173, "OD1"]
a2 = [173, "O"]

rpiHs = [[d1, a1], [d1, a2]]


d1 = [["FAD", 1], "O1", "H_O1"]
d2 = [["FAD", 1], "O11", "H_O11"]
d3 = [["FAD", 1], "N6", "H_N6"]
d4 = [203, "N", "HN"]
d5 = [["FAD", 1], "O5", "H_O5"]
d6 = [["FAD", 1], "O6", "H_O6"]
d7 = [["FAD", 1], "O10", "H_O10"]
d8 = [["FAD", 1], "O12", "H_O12"]

a1 = [203, "OE2"]
a2 = [203, "O"]
a3 = [203, "OE1"]
a4 = [["FAD", 1], "O12"]
a5 = [["FAD", 1], "O5"]
a6 = [["FAD", 1], "O11"]

rpiHs = [[d1, a1], [d2, a2], [d3, a3], [d1, a3], [d3, a1], [d2, a1], [d4, a4], [d4, a5], [d4, a6], [d5, a1], [d6, a3], [d5, a3], [d7, a3], [d6, a1], [d7, a1], [d2, a3], [d8, a3], [d8, a1], [d8, a2]]


d1 = [228, "NE2", "H_NE2"]

a1 = [["FAD", 1], "O6"]
a2 = [["FAD", 1], "O2"]

rpiHs = [[d1, a1], [d1, a2]]


d1 = [174, "OH", "H_OH"]
d2 = [["FAD", 1], "N6", "H_N6"]

a1 = [["FAD", 1], "N2"]
a2 = [174, "OH"]
a3 = [["FAD", 1], "N7"]

rpiHs = [[d1, a1], [d2, a2], [d1, a3]]


d1 = [["FAD", 1], "N6", "H_N6"]

a1 = [167, "OE1"]
a2 = [167, "OE2"]

rpiHs = [[d1, a1], [d1, a2]]


d1 = [206, "OG", "H_OG"]
d2 = [["FAD", 1], "O12", "H_O12"]
d3 = [206, "N", "HN"]
d4 = [["FAD", 1], "O11", "H_O11"]

a1 = [["FAD", 1], "N4"]
a2 = [206, "O"]
a3 = [["FAD", 1], "O11"]

rpiHs = [[d1, a1], [d2, a2], [d3, a1], [d3, a3], [d4, a2]]


d1 = [["FAD", 1], "N6", "H_N6"]

a1 = [177, "OD2"]

rpiHs = [[d1, a1]]


d1 = [132, "NH1", "H_NH1"]
d2 = [132, "NH2", "H_NH2"]

a1 = [["FAD", 1], "O9"]

rpiHs = [[d1, a1], [d2, a1]]


d1 = [204, "N", "HN"]
d2 = [["FAD", 1], "O12", "H_O12"]
d3 = [["FAD", 1], "O11", "H_O11"]

a1 = [["FAD", 1], "O11"]
a2 = [204, "N"]
a3 = [204, "O"]
a4 = [["FAD", 1], "O12"]

rpiHs = [[d1, a1], [d2, a2], [d3, a2], [d3, a3], [d1, a4], [d2, a3]]


d1 = [96, "NE2", "H_NE2"]

a1 = [["FAD", 1], "O13"]
a2 = [["FAD", 1], "N5"]
a3 = [["FAD", 1], "O9"]

rpiHs = [[d1, a1], [d1, a2], [d1, a3]]


d1 = [["FAD", 1], "N3", "H_N3"]

a1 = [160, "O"]

rpiHs = [[d1, a1]]


d1 = [273, "NZ", "H_NZ"]
d2 = [["FAD", 1], "O12", "H_O12"]
d3 = [["FAD", 1], "O11", "H_O11"]
d4 = [["FAD", 1], "O10", "H_O10"]

a1 = [["FAD", 1], "O5"]
a2 = [["FAD", 1], "O6"]
a3 = [["FAD", 1], "O9"]
a4 = [273, "O"]
a5 = [["FAD", 1], "O2"]
a6 = [["FAD", 1], "N6"]
a7 = [["FAD", 1], "N2"]
a8 = [["FAD", 1], "O8"]

rpiHs = [[d1, a1], [d1, a2], [d1, a3], [d2, a4], [d3, a4], [d1, a5], [d1, a6], [d1, a7], [d1, a8], [d4, a4]]


d1 = [["FAD", 1], "O12", "H_O12"]
d2 = [["FAD", 1], "O11", "H_O11"]
d3 = [205, "N", "HN"]

a1 = [205, "O"]
a2 = [["FAD", 1], "N6"]
a3 = [["FAD", 1], "O11"]
a4 = [["FAD", 1], "N4"]
a5 = [["FAD", 1], "N2"]

rpiHs = [[d1, a1], [d2, a1], [d3, a2], [d3, a3], [d3, a4], [d3, a5]]


d1 = [129, "OG1", "H_OG1"]

a1 = [["FAD", 1], "O9"]

rpiHs = [[d1, a1]]


d1 = [272, "OG", "H_OG"]
d2 = [["FAD", 1], "O10", "H_O10"]
d3 = [["FAD", 1], "O11", "H_O11"]
d4 = [["FAD", 1], "O1", "H_O1"]
d5 = [["FAD", 1], "O6", "H_O6"]
d6 = [["FAD", 1], "O12", "H_O12"]

a1 = [["FAD", 1], "O6"]
a2 = [272, "O"]
a3 = [["FAD", 1], "O2"]

rpiHs = [[d1, a1], [d2, a2], [d3, a2], [d1, a3], [d4, a2], [d5, a2], [d6, a2]]


d1 = [68, "NH2", "H_NH2"]
d2 = [68, "NH1", "H_NH1"]

a1 = [["FAD", 1], "O9"]
a2 = [["FAD", 1], "N1"]

rpiHs = [[d1, a1], [d2, a1], [d1, a2]]


d1 = [164, "NE2", "H_NE2"]
d2 = [["FAD", 1], "N6", "H_N6"]
d3 = [164, "N", "HN"]
d4 = [["FAD", 1], "O10", "H_O10"]
d5 = [["FAD", 1], "O11", "H_O11"]

a1 = [["FAD", 1], "N7"]
a2 = [164, "OE1"]
a3 = [164, "NE2"]
a4 = [["FAD", 1], "N2"]
a5 = [["FAD", 1], "N6"]
a6 = [["FAD", 1], "O11"]

rpiHs = [[d1, a1], [d2, a2], [d2, a3], [d3, a1], [d1, a4], [d4, a2], [d1, a5], [d5, a2], [d1, a6]]


d1 = [["FAD", 1], "O12", "H_O12"]
d2 = [["FAD", 1], "O1", "H_O1"]
d3 = [["FAD", 1], "O11", "H_O11"]

a1 = [200, "O"]

rpiHs = [[d1, a1], [d2, a1], [d3, a1]]


d1 = [["FAD", 1], "O12", "H_O12"]
d2 = [["FAD", 1], "O11", "H_O11"]

a1 = [199, "O"]

rpiHs = [[d1, a1], [d2, a1]]


d1 = [165, "N", "HN"]
d2 = [["FAD", 1], "N6", "H_N6"]

a1 = [["FAD", 1], "N6"]
a2 = [165, "O"]

rpiHs = [[d1, a1], [d2, a2]]


d1 = [["FAD", 1], "N6", "H_N6"]

a1 = [162, "O"]

rpiHs = [[d1, a1]]


d1 = [["FAD", 1], "O11", "H_O11"]
d2 = [["FAD", 1], "N6", "H_N6"]

a1 = [163, "OD2"]
a2 = [163, "OD1"]

rpiHs = [[d1, a1], [d2, a1], [d2, a2]]


d1 = [["FAD", 1], "N6", "H_N6"]

a1 = [166, "OE2"]

rpiHs = [[d1, a1]]



#### SAH - Complete Inputs

In [ ]:
### SAH1_ALA461
d1 = [461, "N", "HN"]
d2 = [["SAH", 1], "O3", "H12"]

a1 = [["SAH", 1], "N5"]
a2 = [["SAH", 1], "O4"]
a3 = [461, "O"]

rpiHs = [[d1, a1], [d1, a2], [d2, a3]]




### THR481_SAH1
d1 = [481, "OG1", "HG1"]
d2 = [["SAH", 1], "N4", "H18","H19"]

a1 = [["SAH", 1], "N2"]
a2 = [481, "O"]
a3 = [["SAH", 1], "N4"]
a4 = [481, "OG1"]

rpiHs = [[d1, a1], [d2, a2], [d1, a3], [d2, a4]]




### TYR438_SAH
d1 = [["SAH", 1], "O5", "H16"]
d2 = [["SAH", 1], "N4", "H18","H19"]
d3 = [438, "OH", "HH"]

a1 = [438, "OH"]
a2 = [["SAH", 1], "N5"]

rpiHs = [[d1, a1], [d2, a1], [d3, a2]]




### SAH1_ASN456
d1 = [["SAH", 1], "N1", "H1","H2"]
d2 = [456, "ND2", "HD21", "HD22"]
d3 = [["SAH", 1], "N4", "H18", "H19"]
d4 = [["SAH", 1], "O5", "H16"]

a1 = [456, "ND2"]
a2 = [["SAH", 1], "O1"]
a3 = [["SAH", 1], "S1"]
a4 = [["SAH", 1], "N1"]
a5 = [456, "OD1"]
a6 = [["SAH", 1], "N5"]
a7 = [["SAH", 1], "O5"]

rpiHs = [[d1, a1], [d2, a2], [d2, a3], [d2, a4], [d3, a1], [d3, a5], [d2, a6], [d4, a5], [d2, a7], [d1, a5]]


#### SAH - % for each residue interaction

In [ ]:
SAH1_ALA461: [26.46, 72.42, 45.96]  [{'N_O4': 26.42, 'O3_O': 0.02, 'N_N5': 0.02}, {'N_O4': 72.42}]
THR481_SAH1: [35.65, 71.45, 35.8]  [{'OG1_N2': 31.9, 'N4_O': 3.73, 'OG1_N4': 0.02}, {'OG1_N2': 44.48, 'N4_O': 26.7, 'OG1_N4': 0.17, 'N4_OG1': 0.1}]
TYR438_SAH1: [2.52, 32.5, 29.98]  [{'O5_OH': 0.55, 'N4_OH': 0.17, 'OH_N5': 1.8}, {'OH_N5': 32.48, 'O5_OH': 0.02}]
SAH1_ASN456: [49.47, 22.93, -26.54]  [{'N4_OD1': 26.97, 'ND2_N5': 17.8, 'N4_ND2': 0.15, 'ND2_O1': 3.5, 'ND2_N1': 0.28, 'ND2_O5': 0.15, 'N1_OD1': 0.57, 'N1_ND2': 0.02, 'O5_OD1': 0.03}, {'N4_OD1': 17.15, 'ND2_N5': 2.97, 'ND2_O5': 0.63, 'ND2_O1': 1.48, 'ND2_S1': 0.67, 'O5_OD1': 0.03}]
THR464_SAH1: [58.54, 75.28, 16.74]  [{'O4_OG1': 28.42, 'O3_OG1': 2.95, 'OG1_O4': 26.28, 'OG1_N3': 0.3, 'O4_N': 0.13, 'OG1_N5': 0.02, 'OG1_O3': 0.4, 'O5_OG1': 0.02, 'N_O4': 0.02}, {'O4_OG1': 54.13, 'OG1_O3': 4.23, 'O3_OG1': 15.58, 'OG1_O4': 0.93, 'OG1_N3': 0.08, 'O5_OG1': 0.3, 'OG1_O1': 0.03}]
SAH1_ASN483: [6.16, 18.94, 12.78]  [{'O3_OD1': 5.65, 'ND2_O2': 0.27, 'ND2_S1': 0.12, 'O4_OD1': 0.07, 'N1_OD1': 0.05}, {'ND2_O2': 16.73, 'ND2_S1': 1.15, 'O5_OD1': 0.02, 'ND2_O1': 0.33, 'ND2_O5': 0.48, 'O3_OD1': 0.12, 'ND2_N1': 0.08, 'ND2_N6': 0.03}]
SER484_SAH1: [63.55, 75.93, 12.38]  [{'N_O2': 8.13, 'OG_N3': 13.98, 'O3_O': 9.07, 'O4_O': 30.32, 'O4_OG': 0.1, 'N_N3': 1.4, 'N_O4': 0.25, 'N1_O': 0.3}, {'OG_N3': 44.2, 'N_O2': 27.18, 'O3_O': 4.25, 'N_N3': 0.02, 'N1_O': 0.1, 'O5_O': 0.18}]
SAH1_ASP457: [16.49, 25.3, 8.81]  [{'N_O1': 9.85, 'N_N1': 4.2, 'N_O5': 0.77, 'O5_OD1': 0.77, 'O5_OD2': 0.9}, {'N_O5': 2.3, 'N_N1': 8.47, 'N_O1': 11.65, 'O5_OD2': 0.73, 'O5_OD1': 1.98, 'N1_OD1': 0.1, 'N1_OD2': 0.07}]
SAH1_GLN485: [18.91, 13.5, -5.41]  [{'NE2_O3': 7.67, 'O3_OE1': 5.32, 'NE2_O4': 0.07, 'N1_OE1': 5.78, 'O5_OE1': 0.05, 'O4_OE1': 0.02}, {'N1_OE1': 10.43, 'NE2_O3': 2.12, 'O3_OE1': 0.05, 'O5_OE1': 0.35, 'N1_NE2': 0.13, 'NE2_O5': 0.12, 'NE2_O1': 0.28, 'NE2_N1': 0.02}]
SER369_SAH1: [9.03, 3.8, -5.23]  [{'O5_O': 1.18, 'N1_OG': 1.6, 'O5_OG': 0.3, 'OG_O1': 0.22, 'OG_N1': 1.2, 'N1_O': 4.38, 'OG_S1': 0.03, 'N_N1': 0.12}, {'O5_OG': 0.02, 'OG_O1': 0.13, 'N1_OG': 0.18, 'O5_O': 0.2, 'N1_O': 2.98, 'OG_O5': 0.02, 'OG_N1': 0.08, 'O4_OG': 0.12, 'OG_S1': 0.07}]
THR573_SAH1: [1.04, 4.97, 3.93]  [{'O5_OG1': 0.32, 'OG1_O1': 0.62, 'OG1_N1': 0.1}, {'OG1_O1': 0.58, 'O5_OG1': 0.07, 'OG1_N1': 4.3, 'OG1_O5': 0.02}]
SAH1_PRO348: [12.28, 15.96, 3.68]  [{'O5_O': 7.25, 'N1_O': 5.03}, {'N1_O': 10.43, 'O5_O': 5.53}]
TRP455_SAH1: [0.05, 3.64, 3.59]  [{'N1_O': 0.03, 'O5_O': 0.02}, {'N1_O': 3.62, 'O5_O': 0.02}]
SAH1_GLU463: [17.14, 20.7, 3.56]  [{'O5_OE2': 0.37, 'O4_OE2': 0.35, 'O5_OE1': 0.68, 'O3_OE2': 6.55, 'O4_OE1': 0.52, 'O3_OE1': 6.57, 'O4_O': 0.03, 'N1_OE1': 1.07, 'N1_OE2': 0.68, 'N_O1': 0.32}, {'O3_OE1': 2.77, 'O3_OE2': 4.03, 'N1_OE2': 2.02, 'N1_OE1': 1.87, 'O5_OE1': 2.58, 'O5_OE2': 6.53, 'O4_OE1': 0.55, 'O4_OE2': 0.35}]
SAH1_ARG345: [5.76, 3.55, -2.21]  [{'NH2_N1': 0.02, 'NH1_O1': 2.07, 'NH1_N1': 3.52, 'NH1_O5': 0.08, 'O5_NH1': 0.02, 'N1_NH1': 0.05}, {'NH1_O1': 0.88, 'NH1_O5': 0.13, 'NH2_N1': 0.42, 'NH1_N1': 1.97, 'NH2_O1': 0.15}]
SAH1_ALA368: [0.97, 3.05, 2.08]  [{'N1_O': 0.27, 'O5_O': 0.7}, {'O5_O': 2.65, 'N1_O': 0.4}]
SAH1_ARG370: [0.7, 2.2, 1.5]  [{'NH1_O1': 0.43, 'N1_NH1': 0.15, 'O5_NE': 0.05, 'O5_NH2': 0.07}, {'NH1_N1': 2.05, 'O5_NH1': 0.02, 'NE_N1': 0.13}]
SAH1_PRO459: [0.69, 0.12, -0.57]  [{'O5_O': 0.67, 'N1_O': 0.02}, {'O5_O': 0.12}]
SAH1_ILE365: [0.37, 0, -0.37]  [{'N1_O': 0.37}, {'N/A': 0}]
SAH1_GLU458: [7.99, 7.74, -0.25]  [{'O5_OE1': 0.72, 'O5_OE2': 0.72, 'N1_OE2': 0.93, 'O5_O': 1.98, 'N_O1': 2.45, 'N1_OE1': 0.75, 'N_N1': 0.05, 'N1_O': 0.02, 'N_O5': 0.37}, {'N1_OE2': 0.95, 'N1_OE1': 0.8, 'O5_OE1': 0.5, 'N_N1': 0.08, 'N_O1': 2.23, 'O5_O': 1.7, 'O5_OE2': 0.2, 'N_O5': 1.23, 'O5_N': 0.05}]
THR560_SAH1: [0.12, 0.37, 0.25]  [{'OG1_O5': 0.05, 'O5_OG1': 0.05, 'OG1_O1': 0.02}, {'OG1_O1': 0.32, 'O5_OG1': 0.05}]
SAH1_GLN509: [0.02, 0.13, 0.11]  [{'O5_OE1': 0.02}, {'O5_OE1': 0.08, 'N1_OE1': 0.05}]
SAH1_ASN475: [0, 0.02, 0.02]  [{'N/A': 0}, {'N4_OD1': 0.02}]
SAH1_ALA350: [0.02, 0, -0.02]  [{'N_O1': 0.02}, {'N/A': 0}]
SAH1_LEU347: [0.03, 0.02, -0.01]  [{'O5_O': 0.03}, {'O5_O': 0.02}]
TRP349_SAH1: [0.03, 0.02, -0.01]  [{'O5_N': 0.03}, {'O5_N': 0.02}]

#### SAH - Inputs Where Donor is Missing Hydrogen

In [ ]:
d1 = [461, "N", "HN"]
d2 = [["SAH", 1], "O3", "H_O3"]

a1 = [["SAH", 1], "N5"]
a2 = [["SAH", 1], "O4"]
a3 = [461, "O"]

rpiHs = [[d1, a1], [d1, a2], [d2, a3]]


d1 = [484, "N", "HN"]
d2 = [["SAH", 1], "O4", "H_O4"]
d3 = [["SAH", 1], "N1", "H_N1"]
d4 = [484, "OG", "H_OG"]
d5 = [["SAH", 1], "O5", "H_O5"]
d6 = [["SAH", 1], "O3", "H_O3"]

a1 = [["SAH", 1], "N3"]
a2 = [484, "O"]
a3 = [["SAH", 1], "O4"]
a4 = [["SAH", 1], "O2"]
a5 = [484, "OG"]

rpiHs = [[d1, a1], [d2, a2], [d3, a2], [d4, a1], [d1, a3], [d5, a2], [d1, a4], [d6, a2], [d2, a5]]


d1 = [481, "OG1", "H_OG1"]
d2 = [["SAH", 1], "N4", "H_N4"]

a1 = [["SAH", 1], "N2"]
a2 = [481, "O"]
a3 = [["SAH", 1], "N4"]
a4 = [481, "OG1"]

rpiHs = [[d1, a1], [d2, a2], [d1, a3], [d2, a4]]


d1 = [464, "OG1", "H_OG1"]
d2 = [["SAH", 1], "O3", "H_O3"]
d3 = [["SAH", 1], "O4", "H_O4"]
d4 = [["SAH", 1], "O5", "H_O5"]
d5 = [464, "N", "HN"]

a1 = [["SAH", 1], "N3"]
a2 = [464, "OG1"]
a3 = [["SAH", 1], "O1"]
a4 = [["SAH", 1], "O4"]
a5 = [464, "N"]
a6 = [["SAH", 1], "N5"]
a7 = [["SAH", 1], "O3"]

rpiHs = [[d1, a1], [d2, a2], [d1, a3], [d1, a4], [d3, a5], [d4, a2], [d5, a4], [d3, a2], [d1, a6], [d1, a7]]


d1 = [["SAH", 1], "O5", "H_O5"]

a1 = [349, "N"]

rpiHs = [[d1, a1]]


d1 = [["SAH", 1], "N1", "H_N1"]
d2 = [["SAH", 1], "O5", "H_O5"]

a1 = [348, "O"]

rpiHs = [[d1, a1], [d2, a1]]


d1 = [["SAH", 1], "N1", "H_N1"]
d2 = [456, "ND2", "H_ND2"]
d3 = [["SAH", 1], "N4", "H_N4"]
d4 = [["SAH", 1], "O5", "H_O5"]

a1 = [456, "ND2"]
a2 = [["SAH", 1], "O1"]
a3 = [["SAH", 1], "S1"]
a4 = [["SAH", 1], "N1"]
a5 = [456, "OD1"]
a6 = [["SAH", 1], "N5"]
a7 = [["SAH", 1], "O5"]

rpiHs = [[d1, a1], [d2, a2], [d2, a3], [d2, a4], [d3, a1], [d3, a5], [d2, a6], [d4, a5], [d2, a7], [d1, a5]]


d1 = [["SAH", 1], "O3", "H_O3"]
d2 = [463, "N", "HN"]
d3 = [["SAH", 1], "O4", "H_O4"]
d4 = [["SAH", 1], "N1", "H_N1"]
d5 = [["SAH", 1], "O5", "H_O5"]

a1 = [463, "OE2"]
a2 = [["SAH", 1], "O1"]
a3 = [463, "OE1"]
a4 = [463, "O"]

rpiHs = [[d1, a1], [d2, a2], [d1, a3], [d3, a1], [d3, a4], [d4, a3], [d5, a1], [d5, a3], [d4, a1], [d3, a3]]


d1 = [["SAH", 1], "O5", "H_O5"]
d2 = [["SAH", 1], "N1", "H_N1"]

a1 = [368, "O"]

rpiHs = [[d1, a1], [d2, a1]]


d1 = [573, "OG1", "H_OG1"]
d2 = [["SAH", 1], "O5", "H_O5"]

a1 = [["SAH", 1], "O1"]
a2 = [573, "OG1"]
a3 = [["SAH", 1], "O5"]
a4 = [["SAH", 1], "N1"]

rpiHs = [[d1, a1], [d2, a2], [d1, a3], [d1, a4]]


d1 = [345, "NH1", "H_NH1"]
d2 = [345, "NH2", "H_NH2"]
d3 = [["SAH", 1], "O5", "H_O5"]
d4 = [["SAH", 1], "N1", "H_N1"]

a1 = [["SAH", 1], "N1"]
a2 = [["SAH", 1], "O1"]
a3 = [345, "NH1"]
a4 = [["SAH", 1], "O5"]

rpiHs = [[d1, a1], [d2, a1], [d1, a2], [d3, a3], [d4, a3], [d1, a4], [d2, a2]]


d1 = [458, "N", "HN"]
d2 = [["SAH", 1], "N1", "H_N1"]
d3 = [["SAH", 1], "O5", "H_O5"]

a1 = [["SAH", 1], "O1"]
a2 = [458, "OE2"]
a3 = [458, "OE1"]
a4 = [458, "O"]
a5 = [458, "N"]
a6 = [["SAH", 1], "N1"]
a7 = [["SAH", 1], "O5"]

rpiHs = [[d1, a1], [d2, a2], [d2, a3], [d2, a4], [d3, a5], [d1, a6], [d3, a3], [d3, a2], [d3, a4], [d1, a7]]


d1 = [483, "ND2", "H_ND2"]
d2 = [["SAH", 1], "O3", "H_O3"]
d3 = [["SAH", 1], "O4", "H_O4"]
d4 = [["SAH", 1], "O5", "H_O5"]
d5 = [["SAH", 1], "N1", "H_N1"]

a1 = [["SAH", 1], "O5"]
a2 = [["SAH", 1], "N6"]
a3 = [483, "OD1"]
a4 = [["SAH", 1], "O2"]
a5 = [["SAH", 1], "N1"]
a6 = [["SAH", 1], "O1"]
a7 = [["SAH", 1], "S1"]

rpiHs = [[d1, a1], [d1, a2], [d2, a3], [d1, a4], [d3, a3], [d1, a5], [d4, a3], [d1, a6], [d1, a7], [d5, a3]]


d1 = [["SAH", 1], "N1", "H_N1"]
d2 = [485, "NE2", "H_NE2"]
d3 = [["SAH", 1], "O5", "H_O5"]
d4 = [["SAH", 1], "O4", "H_O4"]
d5 = [["SAH", 1], "O3", "H_O3"]

a1 = [485, "OE1"]
a2 = [["SAH", 1], "O5"]
a3 = [["SAH", 1], "O4"]
a4 = [485, "NE2"]
a5 = [["SAH", 1], "N1"]
a6 = [["SAH", 1], "O1"]
a7 = [["SAH", 1], "O3"]

rpiHs = [[d1, a1], [d2, a2], [d3, a1], [d4, a1], [d2, a3], [d1, a4], [d2, a5], [d2, a6], [d2, a7], [d5, a1]]


d1 = [["SAH", 1], "O5", "H_O5"]
d2 = [369, "OG", "H_OG"]
d3 = [["SAH", 1], "N1", "H_N1"]
d4 = [["SAH", 1], "O4", "H_O4"]
d5 = [369, "N", "HN"]

a1 = [369, "OG"]
a2 = [["SAH", 1], "O5"]
a3 = [["SAH", 1], "O1"]
a4 = [["SAH", 1], "S1"]
a5 = [369, "O"]
a6 = [["SAH", 1], "N1"]

rpiHs = [[d1, a1], [d2, a2], [d2, a3], [d2, a4], [d3, a1], [d1, a5], [d3, a5], [d2, a6], [d4, a1], [d5, a6]]


d1 = [["SAH", 1], "N1", "H_N1"]
d2 = [["SAH", 1], "O5", "H_O5"]

a1 = [459, "O"]

rpiHs = [[d1, a1], [d2, a1]]


d1 = [350, "N", "HN"]

a1 = [["SAH", 1], "O1"]

rpiHs = [[d1, a1]]


d1 = [["SAH", 1], "O5", "H_O5"]
d2 = [["SAH", 1], "N4", "H_N4"]
d3 = [438, "OH", "H_OH"]

a1 = [438, "OH"]
a2 = [["SAH", 1], "N5"]

rpiHs = [[d1, a1], [d2, a1], [d3, a2]]


d1 = [["SAH", 1], "O5", "H_O5"]
d2 = [["SAH", 1], "N1", "H_N1"]

a1 = [455, "O"]

rpiHs = [[d1, a1], [d2, a1]]


d1 = [457, "N", "HN"]
d2 = [["SAH", 1], "O5", "H_O5"]
d3 = [["SAH", 1], "N1", "H_N1"]

a1 = [["SAH", 1], "O1"]
a2 = [457, "OD1"]
a3 = [["SAH", 1], "O5"]
a4 = [["SAH", 1], "N1"]
a5 = [457, "OD2"]

rpiHs = [[d1, a1], [d2, a2], [d1, a3], [d1, a4], [d3, a5], [d3, a2], [d2, a5]]


d1 = [["SAH", 1], "O5", "H_O5"]

a1 = [347, "O"]

rpiHs = [[d1, a1]]


d1 = [560, "OG1", "H_OG1"]
d2 = [["SAH", 1], "O5", "H_O5"]

a1 = [["SAH", 1], "O1"]
a2 = [["SAH", 1], "O5"]
a3 = [560, "OG1"]

rpiHs = [[d1, a1], [d1, a2], [d2, a3]]


d1 = [["SAH", 1], "N1", "H_N1"]

a1 = [365, "O"]

rpiHs = [[d1, a1]]


d1 = [["SAH", 1], "O5", "H_O5"]
d2 = [["SAH", 1], "N1", "H_N1"]
d3 = [370, "NE", "H_NE"]
d4 = [370, "NH1", "H_NH1"]

a1 = [370, "NH2"]
a2 = [370, "NE"]
a3 = [370, "NH1"]
a4 = [["SAH", 1], "N1"]
a5 = [["SAH", 1], "O1"]

rpiHs = [[d1, a1], [d1, a2], [d2, a3], [d1, a3], [d3, a4], [d4, a5], [d4, a4]]


d1 = [["SAH", 1], "N1", "H_N1"]
d2 = [["SAH", 1], "O5", "H_O5"]

a1 = [509, "OE1"]

rpiHs = [[d1, a1], [d2, a1]]


d1 = [["SAH", 1], "N4", "H_N4"]

a1 = [475, "OD1"]

rpiHs = [[d1, a1]]

